geonameid : integer id of record in geonames database
name : name of geographical point (utf8) varchar(200)
asciiname : name of geographical point in plain ascii characters, varchar(200)
alternatenames : alternatenames, comma separated, ascii names automatically transliterated, convenience attribute from alternatename table, varchar(10000)
latitude : latitude in decimal degrees (wgs84)
longitude : longitude in decimal degrees (wgs84)
feature class : see http://www.geonames.org/export/codes.html, char(1)
feature code : see http://www.geonames.org/export/codes.html, varchar(10)
country code : ISO-3166 2-letter country code, 2 characters
cc2 : alternate country codes, comma separated, ISO-3166 2-letter country code, 200 characters
admin1 code : fipscode (subject to change to iso code), see exceptions below, see file admin1Codes.txt for display names of this code; varchar(20)
admin2 code : code for the second administrative division, a county in the US, see file admin2Codes.txt; varchar(80)
admin3 code : code for third level administrative division, varchar(20)
admin4 code : code for fourth level administrative division, varchar(20)
population : bigint (8 byte int)
elevation : in meters, integer
dem : digital elevation model, srtm3 or gtopo30, average elevation of 3''x3'' (ca 90mx90m) or 30''x30'' (ca 900mx900m) area in meters, integer. srtm processed by cgiar/ciat.
timezone : the iana timezone id (see file timeZone.txt) varchar(40)
modification date : date of last modification in yyyy-MM-dd format


In [133]:
import pandas as pd

columns = ['geonameid', 'name', 'asciiname', 'alternatenames',
           'latitude', 'longitude', 'feature_class', 'feature_code',
           'country code', 'cc2', 'admin1_code', 'admin2 code', 'admin3 code',
           'admin4 code', 'population', 'elevation', 'dem', 'timezone', 'modification_date']

df = pd.read_csv('BF.txt', delimiter='\t', header=None)
df = df.set_axis(columns, axis=1, inplace=False)

df.head(5)


,geonameid,name,asciiname,alternatenames,latitude,longitude,feature_class,feature_code,country code,cc2,admin1_code,admin2 code,admin3 code,admin4 code,population,elevation,dem,timezone,modification_date
0,2353158,Zyonguen,Zyonguen,NaN,12.36667,-0.45000,P,PPL,BF,NaN,04,NaN,NaN,NaN,0,NaN,293,Africa/Ouagadougou,2012-06-05
1,2353159,Zyiliwèlè,Zyiliwele,NaN,12.38333,-2.73333,P,PPL,BF,NaN,06,NaN,NaN,NaN,0,NaN,277,Africa/Ouagadougou,2012-06-05
2,2353160,Zyanko,Zyanko,NaN,12.78333,-0.41667,P,PPL,BF,NaN,05,NaN,NaN,NaN,0,NaN,301,Africa/Ouagadougou,2012-06-05
3,2353161,Zouta,Zouta,NaN,13.14908,-1.28197,P,PPL,BF,NaN,05,70.0,NaN,NaN,0,NaN,306,Africa/Ouagadougou,2010-07-31
4,2353162,Zourtenga,Zourtenga,NaN,12.95741,-1.28745,P,PPL,BF,NaN,05,NaN,NaN,NaN,0,NaN,290,Africa/Ouagadougou,2018-09-05


## 3. Opérations de prétraitement et filtres nécéssaires
On gardera uniquement
* Identifiants, Nom de lieux, latitudes et longitudes

In [134]:
keptCols = df[['geonameid', 'name', 'latitude', 'longitude']]
keptCols.head(5)



,geonameid,name,latitude,longitude
0,2353158,Zyonguen,12.36667,-0.45000
1,2353159,Zyiliwèlè,12.38333,-2.73333
2,2353160,Zyanko,12.78333,-0.41667
3,2353161,Zouta,13.14908,-1.28197
4,2353162,Zourtenga,12.95741,-1.28745


* Renommage avec les nom suivant *ID*, *location_name*, *lat*, *long*

In [135]:
colNames = ['ID', 'location_name', 'lat', 'long']
keptCols.columns = colNames
keptCols.head(5)

,ID,location_name,lat,long
0,2353158,Zyonguen,12.36667,-0.45000
1,2353159,Zyiliwèlè,12.38333,-2.73333
2,2353160,Zyanko,12.78333,-0.41667
3,2353161,Zouta,13.14908,-1.28197
4,2353162,Zourtenga,12.95741,-1.28745


* Sauvegarder les données dans un fichier .csv *burkina_location.csv*

In [136]:
keptCols.to_csv('burkina_location.csv')

## 4. Opérations sur le fichier csv *burkina_location.csv*

* Extraire les données contenant le nom *'gounghin'* et l'enregistrer sous *'gounghin.csv'*

In [137]:
burkinaDf = pd.read_csv('burkina_location.csv')
gounghinDf = burkinaDf[burkinaDf['location_name'].str.contains('Gounghin')]
gounghinDf.to_csv('gounghin.csv')
gounghinDf.head(5)



,Unnamed: 0,ID,location_name,lat,long
147,147,2353306,Gounghin,12.06677,-1.42134
7256,7256,2360473,Gounghin,12.62488,-1.36398
10227,10227,2570204,Gounghin,12.31436,-1.37900
10688,10688,10342749,Gounghin,12.06667,-0.15000
10701,10701,10629032,BICIAB // Gounghin,12.35921,-1.54273


* Extraction des sous-parties de la BD(fichier *burkina_location.csv*), dont les premières lettres des noms de lieux sont compris entre 'A' et 'P'.

In [138]:
import string
letters = [x for x in string.ascii_uppercase]
filteredLetters = letters[letters.index('A') : letters.index('Q')]
firstLetterBtwAPDf = burkinaDf[burkinaDf['location_name'].str[0].isin(filteredLetters)]
firstLetterBtwAPDf.head(5)

,Unnamed: 0,ID,location_name,lat,long
131,131,2353290,Forêt Classée de Ziga,12.47106,-1.08644
147,147,2353306,Gounghin,12.06677,-1.42134
314,314,2353473,Dar Salam,12.36146,-1.63909
384,384,2353543,Forêt Classée de Yendéré,10.15000,-5.06667
409,409,2353568,Province du Yatenga,13.58333,-2.41667


In [139]:
firstLetterBtwAPDf.tail(5)

,Unnamed: 0,ID,location_name,lat,long
11290,11290,12358654,Koulholé,13.43898,-1.16817
11291,11291,12358655,Palboa,11.82653,1.65093
11293,11293,12358657,Nagbingou,13.55244,-0.46760
11294,11294,12358665,Niamango,9.82967,-4.33708
11296,11296,12358677,Kiefaye,13.78260,-1.58790


- Identification respective de la latitude, longitude minimale et les noms de lieux correspondants dans le fichier *burkina_location.csv*

In [140]:
#Latitude minimale
burkinaDf['lat'].min()

9.4295

In [141]:
# Nom de lieu
burkinaDf.loc[burkinaDf['lat'] == burkinaDf['lat'].min()]

,Unnamed: 0,ID,location_name,lat,long
11149,11149,12224995,Fadio-Mèpèhn,9.4295,-2.7775


In [142]:
#Longitude minimale
burkinaDf['long'].min()

-5.48333

In [143]:
# Nom de lieu
burkinaDf.loc[burkinaDf['long'] == burkinaDf['long'].min()]

,Unnamed: 0,ID,location_name,lat,long
1255,1255,2354426,Tinobolé,10.75,-5.48333


* Lieux dont les coords sont entre $lat \geq 11$ et $long \leq 0.5$

In [144]:
burkinaDf.loc[(burkinaDf['lat'] >= 11) & (burkinaDf['long'] <= .5)]

,Unnamed: 0,ID,location_name,lat,long
0,0,2353158,Zyonguen,12.36667,-0.45000
1,1,2353159,Zyiliwèlè,12.38333,-2.73333
2,2,2353160,Zyanko,12.78333,-0.41667
3,3,2353161,Zouta,13.14908,-1.28197
4,4,2353162,Zourtenga,12.95741,-1.28745
...,...,...,...,...,...
11288,11288,12358467,Katé,14.13461,-0.81244
11290,11290,12358654,Koulholé,13.43898,-1.16817
11293,11293,12358657,Nagbingou,13.55244,-0.46760
11295,11295,12358676,Sella,14.35699,0.28666


## 5. Sorties Excel

* Créer un fichier Excel nommé *mini_projet*
* Créer une feuille *gounghin* et enregistrer les données contenant le nom *'gounghin'*
* Créer une seconde feuille dans le même fichier, du nom de *'A_to_P'*

In [145]:
with pd.ExcelWriter('mini_projet.xlsx') as writer:
  gounghinDf.to_excel(writer, sheet_name='gounghin')
  firstLetterBtwAPDf.to_excel(writer, sheet_name='A_to_P')